In [2]:
import sys, os, json, weaviate, dspy
from weaviate.classes.init import Auth 
from dspy.retrieve.weaviate_rm import WeaviateRM
from dotenv import load_dotenv
import replicate

# sys.path.append(os.path.join(os.getcwd(), "..","..", "..", "src", "ingestion"))
# from utils.model_loaders import get_embedding

load_dotenv()

True

In [3]:
print(os.environ["WEAVIATE_URL"])

xxxzwsdlfdnshikjxrgcjvubq.c0.us-east1.gcp.weaviate.cloud


In [54]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("DSPy")
mlflow.autolog()

2025/07/08 19:52:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for dspy.
2025/07/08 19:52:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for langchain.
2025/07/08 19:52:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for litellm.
2025/07/08 19:52:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for llama_index.core.
2025/07/08 19:52:37 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.


In [55]:
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WEAVIATE_URL"],
    auth_credentials=Auth.api_key(os.environ["WEAVIATE_API_KEY"]),
)

In [ ]:
question = "Where is northwind substation located?"

In [ ]:
lm = dspy.LM(
    "azure/gpt-4o-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    api_base=os.environ["OPENAI_API_BASE"],
    api_version="2024-12-01-preview",
)

rm = WeaviateRM(
    weaviate_client=client,
    weaviate_collection_name="TextChunk",
    tenant_id="dev",
    weaviate_collection_text_key="text",
)

dspy.configure(lm=lm, rm=rm)

In [58]:
def rerank(query, passages, k=5):
    """Rerank passages using BGE-Reranker-v2-m3"""
    scores = replicate.run(
        "yxzwayne/bge-reranker-v2-m3:7f7c6e9d18336e2cbf07d88e9362d881d2fe4d6a9854ec1260f115cabc106a8c",
        input={"input_list": json.dumps([[query, p] for p in passages])},
    )
    ranked = [passage for score, passage in sorted(zip(scores, passages), reverse=True)]
    return ranked[:k]

In [59]:
class GenerateContextualAnswer(dspy.Signature):
    """Generate an answer to a question based on a set of context chunks"""
    
    context = dspy.InputField(desc="ANN-based context chunks")
    question = dspy.InputField(desc="User question")
    answer = dspy.OutputField(desc="Answer to the question")

class GenerateContextualAnswerWithChatHistory(dspy.Signature):
    """Generate an answer to a question based on a set of context chunks and conversation history"""
    
    chat_history = dspy.InputField(desc="Chat history")
    context = dspy.InputField(desc="ANN-based context chunks")
    question = dspy.InputField(desc="User question")
    answer = dspy.OutputField(desc="Answer to the question")

In [ ]:
class MiniRAG(dspy.Module):
    def __init__(self, k: int = 12):
        self.k = k
        self.reader = dspy.ChainOfThought(GenerateContextualAnswer)

    def forward(self, question: str):
        context = dspy.Retrieve(k=5)(question, vector=get_embedding(question, "text")).passages
        chunks = rerank(question, context)
        return self.reader(context=chunks, question=question).answer

In [61]:
# MIPROv2
# tp = dspy.teleprompt.MIPROv2(metric=dspy.evaluate.SemanticF1(decompositional=True), auto="light", num_threads=4)
# optimized_rag = tp.compile(
#     MiniRAG(),
#     trainset=["",""],
#     max_bootstrapped_demos=0,
#     max_labeled_demos=0,
#     requires_permission_to_run=False
# )

In [62]:
rag = MiniRAG()

result = rag(question)
answer = result.answer if hasattr(result, "answer") else str(result)
print(f"\n💬 {answer}\n")

WeaviateQueryError: Query call with protocol GRPC search failed with message object search at index imagechunk: No properties provided, and no indexed properties found in class.

[Trace(trace_id=de74e3aff58c4e73ab7252f827ed0649), Trace(trace_id=6280ece03276401896ee2eb3e118af23)]